## Generative pretrained image priors for MRI reconstruction
**Creator**: [Guanxiong Luo](https://ggluo.github.io)

**Overview**:
This notebook is to present how to use the pretrained image priors to improve MR reconstruction. What we will use here are the following:
1. a library for generative image prior, called [spreco](https://pypi.org/project/spreco),
2. a MR image reconstruction toolbox, called [BART](https://www.github.com/mrirecon/bart),
3. several pretrained models hosted on this huggingface [page](https://huggingface.co/Guanxiong/MRI-Image-Priors).
4. TensorFlow C API


#### Let's get all of them at first!

In [ ]:
%%bash
git clone https://github.com/ggluo/spreco
cd spreco
git checkout devel
pip install .
pip install huggingface_hub

cd ..
wget -q https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.12.0.tar.gz
mkdir tensorflow && tar -C tensorflow -xvzf libtensorflow-gpu-linux-x86_64-2.12.0.tar.gz

wget -q https://raw.githubusercontent.com/ggluo/image-priors/release/misc/kspace/mprage.hdr
wget -q https://raw.githubusercontent.com/ggluo/image-priors/release/misc/kspace/mprage.cfl

#### Prepare BART

In [ ]:
%env LIBRARY_PATH=/content/tensorflow/include 
%env LD_LIBRARY_PATH=/content/tensorflow/lib
%env TF_CPP_MIN_LOG_LEVEL=3

In [ ]:
%%bash

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Download BART version
[ -d /content/bart ] && rm -r /content/bart
git clone https://github.com/mrirecon/bart/ bart
[ -d "bart" ] && echo "BART was downloaded successfully."

cd bart

COMPILE_SPECS=" PARALLEL=1
                TENSORFLOW=1
                TENSORFLOW_BASE=../tensorflow/"

printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

make &> /dev/null

In [ ]:
%env TOOLBOX_PATH=/content/bart

#### Download models and export the model for BART

In [ ]:
from huggingface_hub import hf_hub_download

smld = ["config.yaml", "smld.data-00000-of-00001", "smld.index", "smld.meta"]
cplx_large = ["config.yaml", "pixelcnn.data-00000-of-00001", "pixelcnn.index", "pixelcnn.meta"]

for file in smld:
    hf_hub_download(repo_id="Guanxiong/MRI-Image-Priors", subfolder="Diffusion/SMLD", filename=file, local_dir="./")

for file in cplx_large:
    hf_hub_download(repo_id="Guanxiong/MRI-Image-Priors", subfolder="PixelCNN/cplx_large", filename=file, local_dir="./")

In [ ]:
from spreco.exporter import exporter
from spreco.common import utils
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

import numpy as np

e = exporter("./PixelCNN/cplx_large", "pixelcnn", default_out=False, path="./PixelCNN/exported", name="cplx_large")

type = '2DCPLX'

if type == '2DMAG':

    x = tf.placeholder(tf.float32, shape=[1, 256, 256, 2], name="input_0")

    x_cplx = tf.complex(x[..., 0], x[..., 1])
    x_mag  = tf.abs(x_cplx)[..., tf.newaxis]

    logits = e.model.eval(x_mag)
    loss   = e.model.loss_func(x_mag, logits) / np.log(2.0) / np.prod([1, 256, 256, 1])


if type == '2DCPLX':

    x = tf.placeholder(tf.float32, shape=[1, 256, 256, 2], name="input_0")

    logits = e.model.eval(x)
    loss   = e.model.loss_func(x, logits) / np.log(2.0) / np.prod([1, 256, 256, 2])


e.export([x], [loss], attach_gradients=True)

#### Perform reconstruction

The below cell includes the following steps
1. create undersampling pattern
2. estimate coil sensitivities
3. compute reference and 
4. reconstruct image with l2-regularization and l1-regularzation in wavelet domain
5. reconstruct image with log-likelihood regularization by a generative prior

In [ ]:
%%bash

nx=256
ny=256
cal=15
fx=3
fy=2

kspace=mprage
graph=./PixelCNN/exported/cplx_large

# create undersampling pattern
bart upat -Y$nx -Z $ny -y $fx -z$fy -c$cal mask
bart transpose 0 1 mask mask
bart transpose 1 2 mask mask
bart fmac mask $kspace und_kspace

# estimate coil sensitivities
bart ecalib -r20 -m1 -c0.001 und_kspace coilsen

# compute reference
bart fft -i 3 $kspace coilimgs
bart fmac -C -s$(bart bitmask 3) coilimgs coilsen grd

# compute zero-filled
bart fft -i 3 und_kspace coilimgs
bart fmac -C -s$(bart bitmask 3) coilimgs coilsen zero_filled

# l2-regularized reconstruction
bart pics -l2 -r 0.01 und_kspace coilsen l2_pics

# l1-regularzied in wavelet domains reconstruction 
bart pics -l1 -r 0.01 und_kspace coilsen l1_pics

# log-likelihood regularized by a generative prior reconstruction
bart pics -i100 -d4 -R TF:{$graph}:0.8 und_kspace coilsen prior_pics

#### Display the reconstruction

In [ ]:
#### Create a function to plot the grid of images

%matplotlib inline
import matplotlib.pyplot as plt
from functools import partial

def subplot(ax, img, title, cmap, interpolation, vmin, vmax):
    ax.imshow(img, cmap=cmap, interpolation=interpolation, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis('off')

plot_params = {'cmap': 'gray', 'interpolation': 'none', 'vmin': 0}
axplot      = partial(subplot, **plot_params)

recos = {"zero_filled": abs(utils.readcfl("zero_filled")), 
         "l2_pics": abs(utils.readcfl("l2_pics")),
         "l1_pics": abs(utils.readcfl("l1_pics")),
         "prior_pics": abs(utils.readcfl("prior_pics")),
         "grd": abs(utils.readcfl("grd"))
        }

grid_x=1
grid_y=5

fig, axss = plt.subplots(grid_x, grid_y, figsize=(15, 15), gridspec_kw={'width_ratios': [1  for _ in range(grid_y)]})
for j, key in enumerate(recos.keys()):
    axplot(axss[j], recos[key], title=key, vmax=0.7*np.max(recos[key]))

#### Generate MR images with the downloaded model
The cell below will take much longer than the above as the generation needs many iterations. Just let it go for around 20mins with a T4 accelerator on Colab, then come back. You will get 25 brain images that have a size of (256,256).

In [ ]:
from spreco.sampler import sampler
import numpy as np

log_path   = './Diffusion/SMLD'
model_path = log_path+'/smld'
config     = utils.load_config(log_path+'/config.yaml')

N=100
sigma_type='quad'
target_snr=0.2
print("INFO -> sigma type: %s, sigma max: %.4f, simga min: %.4f, discrete steps: %d "%( sigma_type, config['sigma_max'], config['sigma_min'], N))

config['N'] = N
a_sampler = sampler(config, target_snr, sigma_type)
a_sampler.init_sampler(model_path, gpu_id='0')

#### Predictor-and-corrector sampling 

In [ ]:
%matplotlib inline
grid_x=5
grid_y=5
samples_n, samples  = a_sampler.pc_sampler(grid_x*grid_y, 3)

images = samples_n[-1]
images = np.abs(utils.float2cplx(images))

fig, axss = plt.subplots(grid_x, grid_y, figsize=(10, 10), gridspec_kw={'width_ratios': [1  for _ in range(grid_x)]})
for i in range(grid_x):
    for j in range(grid_y):
        if i==0:
            strs='x_%d'%j
        else:
            strs=''
        axplot(axss[i,j], images[i*grid_y+j], title=strs, vmax=np.max(images[i*grid_y+j]))
plt.tight_layout(pad=.1)

#### Ancestral sampling

In [ ]:
grid_x=5
grid_y=5
a_sampler.steps=3
samples_n, samples  = a_sampler.ancestral_sampler(grid_x*grid_y, 3)

images = samples[-1]
images = np.abs(utils.float2cplx(images))

fig, axss = plt.subplots(grid_x, grid_y, figsize=(10, 10), gridspec_kw={'width_ratios': [1  for _ in range(grid_x)]})
for i in range(grid_x):
    for j in range(grid_y):
        if i==0:
            strs='x_%d'%j
        else:
            strs=''
        axplot(axss[i,j], images[i*grid_y+j], title=strs, vmax=np.max(images[i*grid_y+j]))
plt.tight_layout(pad=.1)